In [6]:
import sys
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms import QAOA
from qiskit.primitives import Sampler
from qiskit_algorithms.optimizers import COBYLA
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.applications import Knapsack
from qiskit_optimization.algorithms import MinimumEigenOptimizer

In [9]:
def knapsack_argument(total_value_list, total_weight_list, weight_max):
    max_combination_list_idx = []
    
    length = len(total_value_list[0])
    updated_combination_list_idx = [0] * length
    updated_combination_values = total_value_list[0].copy()
    updated_combination_weights = total_weight_list[0].copy()
    
    for i in range(len(total_value_list)-1):
        values = list(total_value_list[i+1][j]- updated_combination_values[j] for j in range(length))
        weights = list(total_weight_list[i+1][j]- updated_combination_weights[j] for j in range(length))
        max_weight = weight_max - sum(updated_combination_weights) # 1 선택을 기본 전제로 깔고, 위 values & weights는 2를 선택했을 때의 상대적 이득을 의미하게 됨
    
        prob = Knapsack(values = values, weights = weights, max_weight = max_weight)
        kqp = prob.to_quadratic_program()
        
        mes = QAOA(sampler=Sampler(), optimizer=COBYLA())
        meo = MinimumEigenOptimizer(min_eigen_solver=mes)
        
        result = meo.solve(kqp)
                
        for j in range(len(result.x)):
            if result.x[j] == 1:
                updated_combination_list_idx[j] = i+1
                updated_combination_values[j] = total_value_list[i+1][j]
                updated_combination_weights[j] = total_weight_list[i+1][j]
    
    return updated_combination_list_idx, updated_combination_values, updated_combination_weights

In [20]:
def formulate_qp(values, weights, weight_max):
    """
    Formulate the knapsack problem as a quadratic program.

    Args:
        values (list[list[float]]): Value list
        weights (list[list[float]]): Weight list
        weight_max (float): Maximum allowable weight.

    Returns:
        QuadraticProgram: The formulated quadratic program.
    """
    
    num_of_comp = len(values[0])
    num_of_markets = len(values) 
    
    qp = QuadraticProgram("Knapsack Problem")

    # Define binary variables
    for i in range(num_of_comp):      
        for j in range(num_of_markets):
            qp.binary_var(name=f"Market{j}_comp{i}")

    # Objective function: Maximize profits
    linear_terms_dict = {}
    for i in range(num_of_comp):
        for j in range(num_of_markets):
            linear_terms_dict[f"Market{j}_comp{i}"] = values[j][i]
            
    qp.maximize(linear=linear_terms_dict)

    # Constraint: Each item must be in exactly one market
    for i in range(num_of_comp):
        qp.linear_constraint(
            linear={f"Market{j}_comp{i}": 1 for j in range(num_of_markets)},
            sense="==",
            rhs=1,
            name=f"state_constraint_{i}",
        )

    # Constraint: Total weight must not exceed wieght_max
    weight_terms_dict = {}
    for i in range(num_of_comp):
        for j in range(num_of_markets):
            weight_terms_dict[f"Market{j}_comp{i}"] = weights[j][i]
    
    qp.linear_constraint(
        linear=weight_terms_dict,
        sense="<=",
        rhs = weight_max,
        name="weight_constraint",
    )

    return qp

def solve_qp_with_qiskit(values, weights, weight_max):
    """
    Solve the quadratic program using Qiskit's optimization solvers.

    Args:
        values (list[list[float]]): Revenue values for markets.
        weights (list[list[float]]): Weight values for markets.
        weight_max (float): Maximum allowable weight.

    Returns:
        dict: Solution details including selected items and total revenue.
    """

    qp = formulate_qp(values, weights, weight_max)

    print(qp.prettyprint())  # Optional: Print the formulated QP

    qaoa = QAOA(sampler = Sampler(), optimizer=COBYLA())
    optimizer = MinimumEigenOptimizer(qaoa)

    result = optimizer.solve(qp)
    solution = result.x

    total_value = 0
    assignments = []

    num_of_comp = len(values[0])
    num_of_markets = len(values)

    for i in range(num_of_comp):
        j = 0
        while j< num_of_markets:
            if solution[j * i + j+1] == 1:
                total_value += values[j][i]
                assignments.append(f"Market{j}_comp{i}")
                break

    return {
        "solution": solution,
        "assignments": assignments,
        "total_value": total_value,
    }


In [ ]:
while True:
    num_of_options = input("How many comparative groups do you want to create?: (press enter if you want to quit) ")
    if num_of_options == "":
        try:
            sys.exit()  # 프로그램 종료
            print(1)  # 실행되지 않음
        except SystemExit:
            pass  # SystemExit 예외를 무시
    if num_of_options.isdigit():  # 숫자인지 확인
        num_of_options = int(num_of_options)
        break
    else:
        print("Please enter a valid number.")

while True:
    num_of_elements = input("How many elements are in each group?: (press enter if you want to quit) ")
    if num_of_elements == "":
        try:
            sys.exit()  # 프로그램 종료
            print(1)  # 실행되지 않음
        except SystemExit:
            pass  # SystemExit 예외를 무시
    if num_of_elements.isdigit():  # 숫자인지 확인
        num_of_elements = int(num_of_elements)
        break
    else:
        print("Please enter a valid number.")
        
while True:
    weight_max = input("What is the maximum allowable weights?: (press enter if you want to quit) ")
    if weight_max == "":
        try:
            sys.exit()  # 프로그램 종료
            print(1)  # 실행되지 않음
        except SystemExit:
            pass  # SystemExit 예외를 무시
    try:
        weight_max = float(weight_max)
        break
    except ValueError:
        print("Please enter a valid number.")


total_value_list = []
total_weight_list = []

for i in range(num_of_options):
    while True:
        values = input(f"Input values for group {i} out of {num_of_options} gropus (separate by spaces, {num_of_elements} values): ")
        value_list = values.split()  # 입력을 공백으로 나눔
        if len(value_list) != num_of_elements:  # 입력 길이 확인
            print(f"Please input exactly {num_of_elements} values.")
            continue
        try:
            value_list = [float(value) for value in value_list]  # 실수 변환
            break
        except ValueError:
            print("All values must be numbers. Please try again.")
            
    while True:
        weights = input(f"Input weights for group {i} out of {num_of_options} gropus (separate by spaces, {num_of_elements} values): ")
        weight_list = weights.split()  # 입력을 공백으로 나눔
        if len(weight_list) != num_of_elements:  # 입력 길이 확인
            print(f"Please input exactly {num_of_elements} weights.")
            continue
        try:
            weight_list = [float(weight) for weight in weight_list]  # 실수 변환
            break
        except ValueError:
            print("All weights must be numbers. Please try again.")

    total_value_list.append(value_list)
    total_weight_list.append(weight_list)

# n번 인덱스는 n번째 비교균을 의미
print("Total value list:", total_value_list)
print("Total weight list:", total_weight_list)
    
# kanpsack argument   
list_combination_idx, combination_values, combination_weights = knapsack_argument(total_value_list,total_weight_list, weight_max)
print(f"values : {combination_values}, weights: {combination_weights}")
print(list_combination_idx)
print("Total value:", sum(combination_values))


print("\n\nThe result above came from qiskit simulatoer.\nNow, we will compare the result with the real quantum computer.")

qcomp_result = solve_qp_with_qiskit(total_value_list, total_weight_list, weight_max)
print("Solution:", qcomp_result["solution"])
print("Assignments:", qcomp_result["assignments"])
print("Total Profit:", qcomp_result["total_profit"])


Total value list: [[3.0, 5.0, 2.0], [7.0, 1.0, 6.0], [8.0, 2.0, 4.0]]
Total weight list: [[2.0, 3.0, 5.0], [4.0, 2.0, 1.0], [3.0, 4.0, 2.0]]


C:\Users\kimhw\AppData\Local\Temp\ipykernel_35368\201519569.py:17: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  mes = QAOA(sampler=Sampler(), optimizer=COBYLA())
C:\Users\kimhw\AppData\Local\Temp\ipykernel_35368\201519569.py:17: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  mes = QAOA(sampler=Sampler(), optimizer=COBYLA())


values : [8.0, 5.0, 6.0], weights: [3.0, 3.0, 1.0]
[2, 0, 1]
Total value: 19.0


The result above came from qiskit simulatoer.
Now, we will compare the result with the real quantum computer.
Problem name: Knapsack Problem

Maximize
  3*Market0_comp0 + 5*Market0_comp1 + 2*Market0_comp2 + 7*Market1_comp0
  + Market1_comp1 + 6*Market1_comp2 + 8*Market2_comp0 + 2*Market2_comp1
  + 4*Market2_comp2

Subject to
  Linear constraints (4)
    Market0_comp0 + Market1_comp0 + Market2_comp0 == 1  'state_constraint_0'
    Market0_comp1 + Market1_comp1 + Market2_comp1 == 1  'state_constraint_1'
    Market0_comp2 + Market1_comp2 + Market2_comp2 == 1  'state_constraint_2'
    2*Market0_comp0 + 3*Market0_comp1 + 5*Market0_comp2 + 4*Market1_comp0
    + 2*Market1_comp1 + Market1_comp2 + 3*Market2_comp0 + 4*Market2_comp1
    + 2*Market2_comp2 <= 10  'weight_constraint'

  Binary variables (9)
    Market0_comp0 Market1_comp0 Market2_comp0 Market0_comp1 Market1_comp1
    Market2_comp1 Market0_comp2 Market1_c

C:\Users\kimhw\AppData\Local\Temp\ipykernel_35368\3116166204.py:73: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  qaoa = QAOA(sampler = Sampler(), optimizer=COBYLA())
